In [156]:
#imports
import re
import nltk
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from tensorflow.keras.preprocessing.text import one_hot

lematizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')


ModuleNotFoundError: No module named 'tensorflow'

In [138]:
#read training dataset
train_data = pd.read_csv('../DATASET/fake-news/train.csv')
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [139]:
#read testing dataset
test_data = pd.read_csv('../DATASET/fake-news/test.csv')
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
train_data['title'].replace('', np.nan, inplace=True)
test_data['title'].replace('', np.nan, inplace=True)

Data Preprocessing

In [140]:
#Function remove unwanted feilds
def removeFeilds(dataset,columns):
    '''
    dataset : Pandas dataset
    columns : columns to be remove in dataset
    '''
    dataset.drop(columns,axis=1,inplace=True)
    dataset.head()

In [141]:
#remove unwanted column in training data
removeFeilds(train_data,['id','author','text'])
train_data.head()

,title,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [142]:
#remove unwanted column in testing data
removeFeilds(test_data,['id','author','text'])
test_data.head()
test_data = test_data['title']

In [143]:
train_data.shape

(20800, 2)

In [144]:
test_data.shape

(5200,)

In [145]:
#check any null value in feilds
print("Training contains null : \n",train_data.isnull().sum())
print("Testing contains null : \n",test_data.isnull().sum())

Training contains null : 
 title    558
label      0
dtype: int64
Testing contains null : 
 122


In [146]:
#remove null values
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [147]:
#check any null value in feilds
print("Training contains null : \n",train_data.isnull().sum())
print("Testing contains null : \n",test_data.isnull().sum())

Training contains null : 
 title    0
label    0
dtype: int64
Testing contains null : 
 0


In [148]:
print(train_data.shape)
print(test_data.shape)

(20242, 2)
(5078,)


In [149]:
# x and y variable
train_x = train_data['title']
train_y = train_data['label']

print(train_x.head())
print(train_y.head())

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object
0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64


In [150]:
#delete unwanted variable
del train_data

In [151]:
#data cleaning
'''
Remove stop words
Remove other than string 
Convert all strings to lower
Convert strings to numeric
'''

'\nRemove stop words\nRemove other than string \nConvert all strings to lower\nConvert strings to numeric\n'

In [152]:
def rootWord(data,isLematize = True):
    '''
    data : Text sentence
    '''

    sentences = nltk.sent_tokenize(data)
    corpus = ''
    
    for i in range(len(sentences)):
        words = sentences[i].split()
        res =[]
        if isLematize:
            print(nltk.pos_tag(words))
            for word, tag in nltk.pos_tag(words):
                wntag = tag[0].lower()
                wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
                lemma = lematizer.lemmatize(word, wntag) if wntag else word
                res.append(lemma)
            
        else:
            res = [stemmer.stem(word) for word in words if not word in set(stopwords.words('english'))]
        
        words = ' '.join(res)
        corpus += words+' '

    return corpus

In [ ]:
def removelink(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

In [153]:
def perprocessing(data):
    '''
    data : Text sentence 
    '''
    # remove non text
    data = re.sub('[^a-zA-Z]', ' ', data)
    #covert to lower
    data = data.lower()
    #split string by space
    data = data.split(' ')
    #remove link
    data = removelink(data)
    #filter if data has space
    data = filter(lambda x : x != '', data)
    #join the list of strings by space
    data = ' '.join(data)
    #find root word of string
    data = rootWord(data,False)
    return data 

In [154]:
for i,data in enumerate(train_x):
    train_x[i] = perprocessing(data)

train_x

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campus breitbart 
2                                    truth might get fire 
3                civilian kill singl us airstrik identifi 
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
19981    nick cannon plan parenthood found extermin bla...
20034    ny daili news near liber news media outlet exp...
20066    david letterman mother endear viewer new york ...
20083          nd avenu subway open jan say new york time 
20196    presidenti golf outing twist trump own place n...
Name: title, Length: 20786, dtype: object

In [155]:
for i,data in enumerate(test_data):
    test_data[i] = perprocessing(data)

test_data

0       specter trump loosen tongu purs string silicon...
1       russian warship readi strike terrorist near al...
2       nodapl nativ american leader vow stay winter f...
3       tim tebow attempt anoth comeback time basebal ...
4                               keiser report meme war e 
                              ...                        
4924    uncensor truth realli happen nodapl protest ye...
4955    new york time reinstat manag editor role appoi...
4961    watch mustang good background show new york time 
4963                        even hillari niec vote trump 
5066    danger polypharmaci ever mount pile pill new y...
Name: title, Length: 5200, dtype: object

In [ ]:
#convert text to numeric
vocab_size = 5000 #how much unique words want to consider for training
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 
onehot_repr

In [168]:
print(removelink("hi iam https://localhost/abb ji"))

hi iam https://localhost/abb ji
hi iam  ji
